In [ ]:
# Instalaciones requeridas
# pip install nbformat
#pip install pandas mysql-connector-python


In [5]:
# 10_bd.ipynb
import sys
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, text
import os

# Configuración del entorno del notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root) if project_root not in sys.path else None
from config import XLSX_DIRECTORY

# Cadena de conexión para SQL Server con autenticación de Windows
connection_string = 'mssql+pyodbc://@DESKTOP-KP6A2O7/NER?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes'
engine = create_engine(connection_string)




# Insertar registros en la tabla ner_tag si no existen previamente
df_ner_tag = pd.read_excel(os.path.join(XLSX_DIRECTORY, "ner_tag.xlsx"))
existing_tags = pd.read_sql('SELECT tag FROM dim_ner_tag', con=engine)
df_ner_tag = df_ner_tag[~df_ner_tag['tag'].isin(existing_tags['tag'])]
df_ner_tag.to_sql('dim_ner_tag', con=engine, if_exists='append', index=False)
print("Datos insertados en la tabla dim_ner_tag.")

# Eliminar registros y resetear identidades de las tablas especificadas
tablas = ['fact_entity_sentiment', 'dim_entity', 'dim_sentiment', 'dim_sentence', 'dim_speech']
for tabla in tablas:
    with engine.connect() as conn:
        trans = conn.begin()
        try:
            conn.execute(text(f"DELETE FROM {tabla}"))
            conn.execute(text(f"DBCC CHECKIDENT ('{tabla}', RESEED, 0)"))
            trans.commit()
            print(f"Registros eliminados y contador de identidad reseteado en la tabla {tabla}.")
        except Exception as e:
            trans.rollback()
            print(f"Error al eliminar registros de la tabla {tabla}: {e}")

# Insertar registros en las tablas speech, sentence, y sentiment
dataframes_y_tablas = [
    ('speech.xlsx', 'dim_speech', 'id_speech'),
    ('sentence.xlsx', 'dim_sentence', 'id_sentence'),
    ('sentiment.xlsx', 'dim_sentiment', 'id_sentiment')
]
for archivo, tabla, columna_id in dataframes_y_tablas:
    try:
        df = pd.read_excel(os.path.join(XLSX_DIRECTORY, archivo))
        if columna_id and columna_id in df.columns:
            df = df.drop(columns=[columna_id])
        df.to_sql(tabla, con=engine, if_exists='append', index=False)
        print(f"Datos insertados en la tabla {tabla}.")
    except Exception as e:
        print(f"Ocurrió un error al procesar el archivo {archivo} para la tabla {tabla}: {e}")


# Insertar registros en la tabla entity_analysis
columnas_entity = ['id_sentence', 'context', 'tag', 'raw_entity', 'entity', 'entity_cluster']
modelos_y_archivos = [
    ('bert_entity.xlsx', 'BERT'),
    ('ner_entity.xlsx', 'NER'),
    ('roberta_entity.xlsx', 'ROBERTA'),
]
for archivo, modelo in modelos_y_archivos:
    df = pd.read_excel(os.path.join(XLSX_DIRECTORY, archivo))
    df = df[columnas_entity]
    df['model'] = modelo
    df.to_sql('dim_entity', con=engine, if_exists='append', index=False)
    print(f"Datos de {modelo} insertados en la tabla entity_analysis.")

Datos insertados en la tabla dim_ner_tag.
Registros eliminados y contador de identidad reseteado en la tabla fact_entity_sentiment.
Registros eliminados y contador de identidad reseteado en la tabla dim_entity.
Registros eliminados y contador de identidad reseteado en la tabla dim_sentiment.
Registros eliminados y contador de identidad reseteado en la tabla dim_sentence.
Registros eliminados y contador de identidad reseteado en la tabla dim_speech.
Datos insertados en la tabla dim_speech.
Datos insertados en la tabla dim_sentence.
Datos insertados en la tabla dim_sentiment.
Datos de BERT insertados en la tabla entity_analysis.
Datos de NER insertados en la tabla entity_analysis.
Datos de ROBERTA insertados en la tabla entity_analysis.
